# Исследование надежности заемщиков.

### Шаг 1. Обзор данных

Составим первое представление о данных.

In [1]:
import pandas as pd
df = pd.read_csv("/datasets/data.csv")
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Шаг 2.1 Заполнение пропусков

В результате просмотра информации о таблице было выявлено два столбца с пропущенными значениями:
* `days_employed` — общий трудовой стаж в днях;
* `total_income` — ежемесячный доход.

Возможной причиной пропуска в данных об общем трудовом стаже может являться утеря трудовой книжки заемщиком. А отсутствие информации о ежемесячном доходе может быть связано с неофициальным видом заработка и невозможностью предоставить справки о доходе.

Ежемесячный доход является основополагающей информацией для оценки надежности заемщика, в связи с этим рекомендуется с таких заемщиков, по которым отсутствует данная информация, запрашивать официальные справки о доходе которые должны быть выданы работодателем.

Оценим долю которую составляют пропущенные значения в каждом из столбцов с пропусками:

In [3]:
share_of_omissions_days_employed = df['days_employed'].isna().mean()
share_of_omissions_total_income = df['total_income'].isna().mean()
print(f'Доля пропущенных значений в столбце "days_employed": {share_of_omissions_days_employed:.1%}')
print(f'Доля пропущенных значений в столбце "total_income": {share_of_omissions_total_income:.1%}')

Доля пропущенных значений в столбце "days_employed": 10.1%
Доля пропущенных значений в столбце "total_income": 10.1%


Доля пропущенных значений не велика и составляет 10.1% от всех данных. Заполнить данные пропуски лучше медианным значением, так как значения количественные, на среднее значение будет сильно влиять максимальные и минимальные значения, и оно может не отражать реальной картины для большинства данных. Медианное же значение делит численные данные "пополам", и ровно половина всех значений будет меньше медианного, а другая половина больше медианного значения.

### Шаг 2.2 Проверка данных на аномалии и исправления.

Значения общего трудового стажа отрицательное, в реальной жизни это невозможно, и данные необходимо исправить. Сделаем данные положительными:

In [4]:
rows = df["days_employed"] < 0
df.loc[rows,"days_employed"] = df["days_employed"]*(-1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
df["days_employed"].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

In [6]:
df[df["days_employed"] >= 365*104].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью


In [7]:
df[df["days_employed"] > 365*104]["days_employed"].count()

3445

In [8]:
df.loc[df["days_employed"] > 365*104, "days_employed"] = 37960

In [9]:
df[df["days_employed"] > 365*104]["days_employed"].count()

0

In [10]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,37960.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Вычислим медианные значения и заполним пропуски:

In [11]:
median_days_employed = df["days_employed"].median()
print(f'Медианное значение в столбце "days_employed": {median_days_employed:.1f}')
df["days_employed"] = df["days_employed"].fillna(median_days_employed)
median_total_income = df["total_income"].median()
print(f'Медианное значение в столбце "total_income": {median_total_income:.1f}')
df["total_income"] = df["total_income"].fillna(median_total_income)

Медианное значение в столбце "days_employed": 2194.2
Медианное значение в столбце "total_income": 145017.9


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Большое количество отрицательных значений в столбце трудового стажа свидетельствует об ошибке в процессе сбора информации. Необходимо сообщить разработчикам. Возможно воспринимаемый знак за минус на самом деле является дефисом.

### Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце 'total_income' и 'days_employed' на целочисленный:

In [13]:
df["total_income"] = df["total_income"].astype("int")
df['days_employed'] = df['days_employed'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Шаг 2.4. Удаление дубликатов.

Удалим явные дубликаты:

In [14]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Определим уникальные значения в столбцах для выявления не явных дубликатов:

In [15]:
df["education"].value_counts()

среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Приведем все значения в столбце к нижнему регистру чтобы исключить не явные дубликаты в столбце:

In [16]:
df["education"] = df["education"].str.lower()

In [17]:
df["family_status"].value_counts()

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Не явные дубликаты в столбце "family_status" отсутствуют.

In [18]:
df["income_type"].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Не явные дубликаты в столбце "income_type" отсутствуют.

In [19]:
df["children"].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Обнаружены ошибки в столбце количество детей: -1 и 20. Заменим -1 на 1 и предположим что в значении 20 был введен ошибочно дополнительный 0, т.е. заменим 20 на 2:

In [20]:
df.loc[df["children"] < 0, "children"] = df["children"]*(-1)
df.loc[df["children"] == 20, "children"] = 2
df["children"].value_counts()

0    14107
1     4856
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [21]:
df["dob_years"].value_counts()

35    616
40    607
41    606
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    538
48    537
37    536
50    513
43    512
32    509
49    508
28    503
45    497
27    493
56    484
52    484
47    477
54    476
46    473
53    459
57    456
58    456
51    448
55    443
59    443
26    408
60    374
25    357
61    354
62    349
63    269
24    264
64    262
23    253
65    194
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце "dob_years" у ста одного заемщика указан возраст 0, заменим его на медианное значение возраста:

In [22]:
median_dob_years = df["dob_years"].median().astype("int")
print(f'Медианное значение в столбце "dob_years": {median_dob_years}')
df.loc[df["dob_years"] == 0, "dob_years"] = median_dob_years

Медианное значение в столбце "dob_years": 42


In [23]:
df["gender"].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

В столбце "gender" указано ошибочное значение XNA. Ввиду того что такое значение всего одно, и из него не ясно к какому типу гендера относится данный заемщик, оставим его без изменений.

Теперь удалим оставшиеся не явные дубликаты:

In [24]:
df = df.drop_duplicates().reset_index(drop=True)

Для удаления явных и не явных дубликатов используется метод .drop_duplicates(). Отличие между явными и не явными дубликатами лишь в том, что повторяющиеся значения повторяются по смыслу, но могут быть записаны по разному, в связи с этим необходима проверка уникальных значений и приведение к единообразной терминологии.

В данном случае причиной дубликатов может быть то, что заемщики заполняют форму самостоятельно и используют свободную терминологию и разный регистр. Для исключения данных дубликатов следует сделать в форме выбор из конкретных значений, без возможности самостоятельного ввода.

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создадим два новых датафрейма, в которых:

* каждому уникальному значению из education соответствует уникальное значение education_id — в первом;
* каждому уникальному значению из family_status соответствует уникальное значение family_status_id — во втором.

In [25]:
df_education_id = df[['education','education_id']]
df_education_id = df_education_id.drop_duplicates().reset_index(drop=True)
df_family_status_id = df[['family_status','family_status_id']]
df_family_status_id = df_family_status_id.drop_duplicates().reset_index(drop=True)

In [26]:
df_education_id

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [27]:
df_family_status_id

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Удалим из исходного датафрейма столбцы "education" и "family_status":

In [28]:
df = df[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,37960,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

Создадим категории доходов:

In [29]:
def total_income_category(income):
    if income <= 30000:
        return "E"
    elif 30000 < income <= 50000 :
        return "D"
    elif 50000 < income <= 200000 :
        return "C"
    elif 200000 < income <= 1000000 :
        return "B"
    else:
        return "A"
df["total_income_category"] = df["total_income"].apply(total_income_category)
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,37960,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


### Шаг 2.7. Категоризация целей кредита.

In [30]:
df["purpose"].value_counts()

свадьба                                   791
на проведение свадьбы                     767
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [31]:
def purpose_category(purpose):
    car_operations = ['на покупку своего автомобиля', 'автомобиль', 'сделка с подержанным автомобилем', 'автомобили', 'свой автомобиль', 'на покупку подержанного автомобиля', 'на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем']
    wedding = ['свадьба', 'на проведение свадьбы', 'сыграть свадьбу']
    education = ['заняться высшим образованием', 'дополнительное образование', 'высшее образование', 'образование', 'получение дополнительного образования', 'получение образования', 'профильное образование', 'получение высшего образования', 'заняться образованием']
    for i in car_operations:
        if purpose == i:
            return "операции с автомобилем"
    for k in wedding:
        if purpose == k:
            return "проведение свадьбы"
    for l in education:
        if purpose == l:
            return "получение образования"
    return "операции с недвижимостью"
df["purpose_category"] = df["purpose"].apply(purpose_category)
df.head(5)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,37960,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


### Ответы на вопросы.

##### Вопрос 1:

Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [32]:
df_pivot_children = df.pivot_table(index='children', values='debt', aggfunc= ['sum', 'count'])
df_pivot_children.columns = ['debt_sum', 'debt_count']
df_pivot_children['share'] = df_pivot_children["debt_sum"] / df_pivot_children["debt_count"]
df_pivot_children = df_pivot_children.sort_values('share' , ascending = False)
df_pivot_children

,debt_sum,debt_count,share
children,,,
4,4,41,0.097561
2,202,2128,0.094925
1,445,4855,0.091658
3,27,330,0.081818
0,1063,14090,0.075444
5,0,9,0.000000


##### Вывод 1:

С увеличением количества детей видно увеличение количества просроченных задолженностей. Однако, люди с 3 детьми платят чаще чем люди с 1 ребенком, возможно выборка слишком мала для достаточного анализа. Люди без детей просрачивают оплату по кредиту гораздо реже, чем люди с детьми и являются надежными заемщиками.

##### Вопрос 2:

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
df_pivot_family_status = df.pivot_table(index='family_status_id', values='debt', aggfunc= ['sum', 'count'])
df_pivot_family_status.columns = ['debt_sum', 'debt_count']
df_pivot_family_status['share'] = df_pivot_family_status["debt_sum"] / df_pivot_family_status["debt_count"]
df_pivot_family_status = df_pivot_family_status.sort_values('share' , ascending = False)
df_pivot_family_status

,debt_sum,debt_count,share
family_status_id,,,
4,274,2810,0.097509
1,388,4150,0.093494
0,931,12339,0.075452
3,85,1195,0.071130
2,63,959,0.065693


##### Вывод 2:

Зависимость между семейным положением и возвратом кредита в срок имеется. Люди не в браке имеют большой процент невозвратов в срок. А те кто развелись или овдовели платят в срок чаще чем люди в браке.

##### Вопрос 3:

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
df_pivot_income_category = df.pivot_table(index='total_income_category', values='debt', aggfunc= ['sum', 'count'])
df_pivot_income_category.columns = ['debt_sum', 'debt_count']
df_pivot_income_category['share'] = df_pivot_income_category["debt_sum"] / df_pivot_income_category["debt_count"]
df_pivot_income_category = df_pivot_income_category.sort_values('share' , ascending = False)
df_pivot_income_category

,debt_sum,debt_count,share
total_income_category,,,
E,2,22,0.090909
C,1360,16015,0.084920
A,2,25,0.080000
B,356,5041,0.070621
D,21,350,0.060000


##### Вывод 3:

Явной зависимости между уровнем дохода и возвратом кредита в срок не обнаружено. Возможно это связано с разницей в сумме заема, для разных заемщиков с разным уровнем дохода. Чем больше уровень дохода, тем больше берется сумма заема. В итоге долговая нагрузка одинакова.

##### Вопрос 4:

Как разные цели кредита влияют на его возврат в срок?

In [35]:
df_pivot_purpose_category = df.pivot_table(index='purpose_category', values='debt', aggfunc= ['sum', 'count'])
df_pivot_purpose_category.columns = ['debt_sum', 'debt_count']
df_pivot_purpose_category['share'] = df_pivot_purpose_category["debt_sum"] / df_pivot_purpose_category["debt_count"]
df_pivot_purpose_category = df_pivot_purpose_category.sort_values('share' , ascending = False)
df_pivot_purpose_category

,debt_sum,debt_count,share
purpose_category,,,
операции с автомобилем,403,4306,0.093590
получение образования,370,4013,0.092200
проведение свадьбы,186,2323,0.080069
операции с недвижимостью,782,10811,0.072334


##### Вывод 4:

Больше всего процент просрочек у тех, кто берет кредит на автомобили и образование. Ипотечное кредитование остается самым надежным.

## Общий вывод:

Семейный статус влияет на просрочку платежа - люди которые были в браке чаще плятят в срок чем те, кто не бывали в браке, такие люди ответственно подходят к обеспечению своей семьи. Однако, с другой стороны, чем больше детей, тем чаще просрочки платежей - прежде всего это может быть связано с непредвиденными расходами на детей, а также уменьшающимся подушевого дохода рассчитанного на каждого члена семьи.

Результаты исследования:
    
1. Были обнаружены пропущенные значения в столбцах 'days_employed' и 'total_income'. Пропуски составляют 10.1% от всей информации в соответствующих столбцах и были заполнены медианными значениями;
2. Были обнаружены аномалии:
    1. в столбце 'days_employed' большое количество отрицательных значений и выбросов - отрицательные значения были изменены на положительные, а выбросы были предварительно приведены к принятому в расчете значению;
    2. Обнаружены аномалии в столбцах 'children', 'dob_years' и 'gender'. Аномалии в столбцах 'children' и 'dob_years' были исправлены.
3. Были удалены дубликаты строк, как явные так и неявные.
4. Была проведена категоризация потенциальных заемщиков по доходу и цели кредита.
5. Были проверены основные предположения о взаимосвязях между характеристиками потенциальных заемщиков и их просрочкой по кредиту на предоставленных данных, по итогу был сделан общий вывод о взаимосвязях:
    
Семейный статус влияет на просрочку платежа - люди которые были в браке чаще плятят в срок чем те, кто не бывали в браке, такие люди ответственно подходят к обеспечению своей семьи. Однако, с другой стороны, чем больше детей, тем чаще просрочки платежей - прежде всего это может быть связано с непредвиденными расходами на детей, а также уменьшающимся подушевого дохода рассчитанного на каждого члена семьи.